The KNN algorithm is very simple to implement, as it does not need to be trained. The training phase merely stores the training data. For each test point, we calculate the distance of that data point to every existing data point and find the $K$ closest ones. What we return is the the most common amongst the top k classification nearest to the test point. Here's the pseudocode for _K_ Nearest Neighbors:

```
kNN:

    Learn:
        Store training set T to X_train: X_train <-- T


    Predict:
        for every point xp in X_predict:
            for every point x in X_train:
                calculate the distance d in D between x and xp
            sort D in increasing order
            take the "k" items in X_train with the smallest distances to x
            return the majority class among these k items
```

Note that for large data sets, the algorithm can take very long to classify because it has to calculate the distance between the test point and evey other point in the data!


# Distance Metrics - Note

As explained above, KNN assigns a class to the test point based on the majority class of  KK  nearest neighbours. In general, euclidean distance is used to find nearest neighbours, but other distance metrics can also be used.

As the dimensionality of the feature space increases, the euclidean distance often becomes problematic due to the curse of dimensionality (discussed later).

In such cases, alternative vector-based similarity measures (dot product, cosine similarity, etc) are used to find the nearest neighbours. This transforms the original metric space into one more amicable to point-to-point measurements.

Another distance measure that you might consider is Mahalanobis distance. Here's the wikipedia page on Mahalanobis distance. https://en.wikipedia.org/wiki/Mahalanobis_distance Mahalanobis distance attempts to weight features according to their probabilities. On some data sets that may be important.

In general, it's probably a good idea to normalize the data at a minimum. Here's a link to the scikit learn scaling package. http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html . You have to be a little circumspect about employing any technique where the answers change with scaling.

In [1]:
reset -fs

In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics.pairwise import paired_distances
from sklearn.metrics.pairwise import euclidean_distances
from statistics import mean as mean
import operator
from statistics import mode as mode
from sklearn.metrics import confusion_matrix
from scipy.spatial import distance
import statistics


In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Assign colum names to the dataset
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# Read dataset to pandas dataframe
dataset = pd.read_csv(url, names=names)
dataset.head()

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
h = []
for i in dataset.Class:
    if i == 'Iris-setosa':
        h.append(0)
    elif i == 'Iris-versicolor':
        h.append(1)
    elif i == 'Iris-virginica':
        h.append(2)
        

In [5]:

dataset["Class"]=h
dataset.tail()

,sepal-length,sepal-width,petal-length,petal-width,Class
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2
149,5.9,3.0,5.1,1.8,2


In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
import statistics
def find_min_mode(list1):
    list_table = statistics._counts(list1)
    len_table = len(list_table)

    if len_table == 1:
        max_mode = statistics.mode(list1)
    else:
        new_list = []
        for i in range(len_table):
            new_list.append(list_table[i][0])
        max_mode = min(new_list) # use the min value here
    return max_mode


#from https://stackoverflow.com/questions/50959614/no-unique-mode-found-2-equally-common-values

In [10]:
def row(X_train, i, k, metric='euclidean'):


    

    D=[]
    for indx, x in enumerate(X_train):
        x=x.reshape(-1, 1)
        #print(x)
        d = distance.euclidean(i,x)#.mean()
        #print("d:",d)
        D.append((indx, d))


        
    D.sort(key=operator.itemgetter(1))
    sort = []
    for j in range(k):
        sort.append(D[j][0])
    #print(sort)
    return sort

In [11]:
def knn(X_train, X_predict, k, metric='euclidean'):
    pred=[]
    out=[]
    listing=[]
    for i in X_predict:
        i=i.reshape(-1, 1)
        #print(i)
        out.append(row(X_train, i, k))

    for j in range(len(out)):
        #print(y_train[out[j]])
        #pred.append(round(max(out[j])))
        listing.append(find_min_mode(y_train[out[j]]))    
        
    #for f in pred:
        #listing.append(y_train[f])

        
    return np.array(listing)


In [12]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x] == predictions[x]:
            correct += 1
    return (correct/int(len(testSet))) * 100.0

In [13]:
getAccuracy(y_test, knn(X_train,X_test,3))

93.33333333333333

In [14]:
y_test

array([0, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 0, 0, 2, 1, 2, 1,
       2, 2, 1, 2, 1, 2, 2, 0])

In [15]:
knn(X_train,X_test,3)

array([0, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 0, 0, 2, 1, 1, 1,
       2, 2, 1, 2, 1, 2, 1, 0])

In [16]:
confusion_matrix(y_test, knn(X_train,X_test,8))

array([[ 8,  0,  0],
       [ 0, 10,  0],
       [ 0,  2, 10]])

In [17]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=8)
neigh.fit(X_train, y_train) 
y_pred = neigh.predict(X_test)


print(confusion_matrix(y_test, y_pred))

[[ 8  0  0]
 [ 0 10  0]
 [ 0  2 10]]


In [18]:
getAccuracy(y_test, y_pred)

93.33333333333333